<a href="https://colab.research.google.com/github/V-Nayak/ML/blob/main/music_recom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import heapq
from scypy import stats

class MusicRecommender:
    def __init__(self, user_data, song_features):

        #Initialize the recommender with user data and song features

        #Parameters:
        # user_data: DataFrame with columns [User_ID, Song_ID, Rating]
        # song_features: DataFrame with columns [Song_ID, Feature_1, Feature_2, ...]

        self.user_data = user_data
        self.song_features = song_features

    def calculate_cosine_similarity(self, song1, song2):

        #Calculate cosine similarity between two songs

        #Parameters:
        #- song1: Feature vector of first song
        #- song2: Feature vector of second song

        #Returns:
        #- Cosine similarity score

        # Normalize feature vectors
        song1_norm = song1 / np.linalg.norm(song1)
        song2_norm = song2 / np.linalg.norm(song2)

        # Calculate cosine similarity
        return np.dot(song1_norm, song2_norm)

    def recommend_songs(self, user_id, top_n=5):

        #Recommend top N songs for a given user

        #Parameters:
        #- user_id: ID of the user to recommend songs for
        #- top_n: Number of recommendations to return

        #Returns:
        #- List of recommended song IDs

        # Get user's highest-rated songs
        user_songs = self.user_data[self.user_data['User_ID'] == user_id]
        top_rated_songs = user_songs.nlargest(3, 'Rating')['Song_ID'].tolist()

        # Calculate song similarities
        recommendations = {}
        for song_id in top_rated_songs:
            # Get features of the top-rated song
            song_features = self.song_features[self.song_features['Song_ID'] == song_id].iloc[0, 1:].values

            # Compare with all other songs
            for _, other_song in self.song_features.iterrows():
                if other_song['Song_ID'] not in user_songs['Song_ID'].values:
                    other_features = other_song.values[1:]
                    similarity = self.calculate_cosine_similarity(song_features, other_features)
                    recommendations[other_song['Song_ID']] = max(
                        recommendations.get(other_song['Song_ID'], 0),
                        similarity
                    )

        # Return top N recommendations
        return sorted(recommendations, key=recommendations.get, reverse=True)[:top_n]